EDA для метаданных

In [1]:
import pandas as pd
import numpy as np
from typing import Dict, List, Optional, Set
from collections import Counter
import Levenshtein
import matplotlib.pyplot as plt
from IPython.core.oinspect import get_encoding
from category_encoders import CatBoostEncoder
from src import future_extract as fe
from src import text_model as tm
import matplotlib as mpl
import seaborn as sns


from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
mpl.style.use('ggplot')
sns.set_style('white')
from src import preproces_text as pt
import importlib
importlib.reload(pt)

/Users/vladislavdemin/PycharmProjects/scientificProject/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<module 'src.preproces_text' from '/Users/vladislavdemin/PycharmProjects/BB-E-CUP/src/preproces_text.py'>

In [2]:
path_train = '/Users/vladislavdemin/Desktop/ml_hack/ml_ozon_сounterfeit_train.csv'
path_test = '/Users/vladislavdemin/Desktop/ml_hack/ml_ozon_сounterfeit_test.csv'

metadata_raw = pd.read_csv('/Users/vladislavdemin/Desktop/ml_hack/processed_data.csv')
metadata_val = pd.read_csv('/Users/vladislavdemin/Desktop/ml_hack/processed_datatest.csv')

In [28]:
data_val2 = pd.read_csv('/Users/vladislavdemin/Desktop/ml_hack/ml_ozon_сounterfeit_test.csv')

In [3]:
data_raw, data_val = pt.full_preprocess(path_train, path_test)

Initialized TF-IDF + SVD (fast mode)
Processing DataFrame with 197198 rows
Text column: description
Encoding texts using fast strategy...


Encoding batches: 100%|██████████| 1972/1972 [00:30<00:00, 64.71it/s]


Created 100 embedding columns
Final DataFrame shape: (197198, 148)
Processing DataFrame with 22760 rows
Text column: description
Encoding texts using fast strategy...


Encoding batches: 100%|██████████| 228/228 [00:03<00:00, 61.90it/s]


Created 100 embedding columns
Final DataFrame shape: (22760, 147)
Processing DataFrame with 197198 rows
Text column: name_rus
Encoding texts using fast strategy...


Encoding batches: 100%|██████████| 198/198 [00:04<00:00, 44.83it/s]


Created 100 embedding columns
Final DataFrame shape: (197198, 120)
Processing DataFrame with 22760 rows
Text column: name_rus
Encoding texts using fast strategy...


Encoding batches: 100%|██████████| 23/23 [00:00<00:00, 38.98it/s]


Created 100 embedding columns
Final DataFrame shape: (22760, 120)


In [4]:
metadata_raw = metadata_raw.drop(['low_price_flag', 'ItemID', "name_rus","has_name","has_category","high_price_flag", 'id'],axis=1)
metadata_val = metadata_val.drop(['low_price_flag', 'ItemID', "name_rus","has_name","has_category","high_price_flag", 'id'],axis=1)

metadata_raw['resolution'] = data_raw['resolution']

metadata_raw_clean, metadata_val_clean = pt.clean_small_brands(metadata_raw, metadata_val)

In [5]:
data_raw1 = data_raw.drop(['brand_name', 'CommercialTypeName4', 'resolution'], axis=1)
data_val1 = data_val.drop(['brand_name', 'CommercialTypeName4', ], axis=1)

conecting_data_raw = pd.concat([metadata_raw_clean ,data_raw1], axis=1) 
conecting_data_val = pd.concat([metadata_val_clean ,data_val1], axis=1) 

In [6]:
X = conecting_data_raw.drop('resolution', axis=1)
y = conecting_data_raw['resolution']  # целевая колонка


# train (70%), test (15%), val (15%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

X_train = pd.concat([X_train, X_val])
y_train = pd.concat([y_train, y_val])

X_train['resolution'] = y_train

target_col = "resolution"  # укажи свою целевую колонку
cat_cols = ["brand_name", "CommercialTypeName4"]  # какие категориальные кодируем
cbe = CatBoostEncoder(cols=cat_cols, random_state=42, handle_unknown='ignore')

cbe.fit(X_train[cat_cols], X_train[target_col])

X_train_enc = X_train.copy()
X_train_enc[cat_cols] = cbe.transform(X_train[cat_cols])

X_train_enc = X_train_enc.drop('resolution', axis=1)

XGB_model1 = XGBClassifiermodel = XGBClassifier(
    n_estimators=120,
    max_depth=10,  # Уменьшаем глубину
    learning_rate=0.07587945476302646,  # Меньший learning rate
    min_child_weight=3,  # Увеличиваем для борьбы с переобучением
    subsample=1.0,  # Сэмплирование данных
    colsample_bytree=0.7,  # Сэмплирование признаков
    gamma=0.1,  # Минимальное уменьшение loss для разделения
    reg_alpha=0.01,  # L1 регуляризация
    reg_lambda=0.1,  # L2 регуляризация
    random_state=42,
    scale_pos_weight=(len(y_train[y_train==0]) / len(y_train[y_train==1])*0.3)# более мягкий вариант
)

In [7]:
XGB_model1.fit(X_train_enc, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=0.1, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.07587945476302646, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=120, n_jobs=None,
              num_parallel_tree=None, ...)

In [8]:
X_test['resolution'] = y_test

target_col = "resolution"  # укажи свою целевую колонку
cat_cols = ["brand_name", "CommercialTypeName4"]  # какие категориальные кодируем

X_test_enc = X_test.copy()
X_test_enc[cat_cols] = cbe.transform(X_test[cat_cols])

X_test_enc = X_test_enc.drop('resolution', axis=1)

In [9]:
threshold = 0.6
y_proba = XGB_model1.predict_proba(X_test_enc)[:, 1]
y_pred = (y_proba >= threshold).astype(int)
accuracy_RFC_model = accuracy_score(y_test, y_pred),
report_RFC_model = classification_report(y_test, y_pred)
print(accuracy_RFC_model)
print(report_RFC_model)

(0.9760649087221095,)
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     27622
           1       0.81      0.83      0.82      1958

    accuracy                           0.98     29580
   macro avg       0.90      0.91      0.90     29580
weighted avg       0.98      0.98      0.98     29580



In [ ]:
conecting_data_val_enc = conecting_data_val.copy()
conecting_data_val_enc[cat_cols] = cbe.transform(conecting_data_val_enc[cat_cols])

In [ ]:
val_prediction = XGB_model1.predict(conecting_data_val_enc)
val_prediction.sum()

In [10]:
imp = pd.Series(XGB_model1.feature_importances_, index=X_train_enc.columns).sort_values(ascending=False)
top_features = imp.head(300).index  # оставить топ-500

In [11]:
X = conecting_data_raw.drop('resolution', axis=1)
y = conecting_data_raw['resolution']  # целевая колонка


# train (70%), test (15%), val (15%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

X_train = pd.concat([X_train, X_val])
y_train = pd.concat([y_train, y_val])

X_train['resolution'] = y_train

target_col = "resolution"  # укажи свою целевую колонку
cat_cols = ["brand_name", "CommercialTypeName4"]  # какие категориальные кодируем
cbe = CatBoostEncoder(cols=cat_cols, random_state=42, handle_unknown='ignore')

cbe.fit(X_train[cat_cols], X_train[target_col])

X_train_enc = X_train.copy()
X_train_enc[cat_cols] = cbe.transform(X_train[cat_cols])

X_train_enc = X_train_enc.drop('resolution', axis=1)
X_train_enc = X_train_enc[top_features]

XGB_model2 = XGBClassifiermodel = XGBClassifier(
    n_estimators=120,
    max_depth=10,  # Уменьшаем глубину
    learning_rate=0.07587945476302646,  # Меньший learning rate
    min_child_weight=3,  # Увеличиваем для борьбы с переобучением
    subsample=1.0,  # Сэмплирование данных
    colsample_bytree=0.7,  # Сэмплирование признаков
    gamma=0.1,  # Минимальное уменьшение loss для разделения
    reg_alpha=0.01,  # L1 регуляризация
    reg_lambda=0.1,  # L2 регуляризация
    random_state=42,
    scale_pos_weight=(len(y_train[y_train==0]) / len(y_train[y_train==1])*0.3)# более мягкий вариант
)

In [12]:
XGB_model2.fit(X_train_enc, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=0.1, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.07587945476302646, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=120, n_jobs=None,
              num_parallel_tree=None, ...)

In [13]:
X_test['resolution'] = y_test

target_col = "resolution"  # укажи свою целевую колонку
cat_cols = ["brand_name", "CommercialTypeName4"]  # какие категориальные кодируем

X_test_enc = X_test.copy()
X_test_enc[cat_cols] = cbe.transform(X_test[cat_cols])

X_test_enc = X_test_enc.drop('resolution', axis=1)
X_test_enc = X_test_enc[top_features]

In [21]:
threshold = 0.6
y_proba = XGB_model2.predict_proba(X_test_enc)[:, 1]
y_pred = (y_proba >= threshold).astype(int)
accuracy_RFC_model = accuracy_score(y_test, y_pred),
report_RFC_model = classification_report(y_test, y_pred)
print(accuracy_RFC_model)
print(report_RFC_model)

(0.9767748478701825,)
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     27622
           1       0.82      0.84      0.83      1958

    accuracy                           0.98     29580
   macro avg       0.90      0.91      0.91     29580
weighted avg       0.98      0.98      0.98     29580



In [22]:
conecting_data_val_enc = conecting_data_val.copy()
conecting_data_val_enc[cat_cols] = cbe.transform(conecting_data_val_enc[cat_cols])

In [25]:
val_prediction_proba = XGB_model1.predict_proba(conecting_data_val_enc)[:, 1]
val_prediction = (val_prediction_proba >= threshold).astype(int)

val_prediction.sum()

np.int64(1938)

In [27]:
data_val['id']

KeyError: 'id'

In [29]:
submission = pd.DataFrame({
    'id': data_val2['id'],
    'prediction': val_prediction
    
})
submission.to_csv('/Users/vladislavdemin/Desktop/ml_hack/submission1.csv', index=False)
dddd = pd.read_csv('/Users/vladislavdemin/Desktop/ml_hack/submission1.csv')
dddd['prediction'].sum()

np.int64(1938)